In [1]:
import pymongo
import pandas as pd

In [2]:
game_dict= { 'NASA': {'user1':10,
                    'user2':7,
                    'user3':2},
           'STAR WARS': {'user2':5,
                        'user4':10,
                        'user5':2},
            'Mario': {'user1': 10,
                     'user3':8,
                     'user6':2}}

In [3]:
game_dict['STAR WARS']

{'user2': 5, 'user4': 10, 'user5': 2}

In [4]:
game_dict

{'NASA': {'user1': 10, 'user2': 7, 'user3': 2},
 'STAR WARS': {'user2': 5, 'user4': 10, 'user5': 2},
 'Mario': {'user1': 10, 'user3': 8, 'user6': 2}}

In [5]:
def flatten_game_dict(game_dict):
    for (game_id, user_score_dict) in game_dict.items():
        for (username, score) in user_score_dict.items():
            yield {'game_id': game_id, 'username': username, 'score': score}
            

In [6]:
flatten_game_dict(game_dict=game_dict)

<generator object flatten_game_dict at 0x110b44bf8>

In [7]:
mc=pymongo.MongoClient()

In [8]:
db=mc['practice_db']

In [9]:
practice=db['practice']

flattened = flatten_game_dict(game_dict=game_dict)

#for thing in flattened:
    practice.insert_one(thing)

In [10]:
practice.find_one()

{'_id': ObjectId('5c0964d7933dea443f61b922'),
 'game_id': 'NASA',
 'username': 'user1',
 'score': 10}

In [11]:
def load_reviews_for_game(key, game_id):
    found_reviews = practice.find({key: game_id})
    return found_reviews

In [12]:
load_reviews_for_game('username', 'user1').distinct('game_id')

['NASA', 'Mario']

In [30]:
df=pd.DataFrame(list(practice.find()))

In [14]:
df

,_id,game_id,score,username
0,5c0964d7933dea443f61b922,NASA,10,user1
1,5c0964d7933dea443f61b923,NASA,7,user2
2,5c0964d7933dea443f61b924,NASA,2,user3
3,5c0964d7933dea443f61b925,STAR WARS,5,user2
4,5c0964d7933dea443f61b926,STAR WARS,10,user4
5,5c0964d7933dea443f61b927,STAR WARS,2,user5
6,5c0964d7933dea443f61b928,Mario,10,user1
7,5c0964d7933dea443f61b929,Mario,8,user3
8,5c0964d7933dea443f61b92a,Mario,2,user6


In [15]:
for use in df.username.unique():
    print(use)

user1
user2
user3
user4
user5
user6


In [16]:
df[df.username=='user1']

,_id,game_id,score,username
0,5c0964d7933dea443f61b922,NASA,10,user1
6,5c0964d7933dea443f61b928,Mario,10,user1


In [17]:
preferences=df.pivot(index='username', columns='game_id', values='score' )

In [18]:
preferences

game_id,Mario,NASA,STAR WARS
username,,,
user1,10.0,10.0,NaN
user2,NaN,7.0,5.0
user3,8.0,2.0,NaN
user4,NaN,NaN,10.0
user5,NaN,NaN,2.0
user6,2.0,NaN,NaN


In [19]:
users=set(df['username'])
games=set(df['game_id'])

In [20]:
game_id_lookup = dict(zip(games, range(len(games))))
user_id_lookup = dict(zip(users, range(len(users))))

In [21]:
df['game_number']=df['game_id'].apply(game_id_lookup.get)
df['user_number']=df['username'].apply(user_id_lookup.get)

In [22]:
df

,_id,game_id,score,username,game_number,user_number
0,5c0964d7933dea443f61b922,NASA,10,user1,1,1
1,5c0964d7933dea443f61b923,NASA,7,user2,1,2
2,5c0964d7933dea443f61b924,NASA,2,user3,1,4
3,5c0964d7933dea443f61b925,STAR WARS,5,user2,2,2
4,5c0964d7933dea443f61b926,STAR WARS,10,user4,2,5
5,5c0964d7933dea443f61b927,STAR WARS,2,user5,2,3
6,5c0964d7933dea443f61b928,Mario,10,user1,0,1
7,5c0964d7933dea443f61b929,Mario,8,user3,0,4
8,5c0964d7933dea443f61b92a,Mario,2,user6,0,0


In [23]:
preferences=df.pivot(index='user_number', columns='game_number', values='score' )

In [24]:
preferences

game_number,0,1,2
user_number,,,
0,2.0,NaN,NaN
1,10.0,10.0,NaN
2,NaN,7.0,5.0
3,NaN,NaN,2.0
4,8.0,2.0,NaN
5,NaN,NaN,10.0


This is practicing to drop rows given a certain criteria of number of reviews for a game or number of reviews a user has given

In [25]:
df

,_id,game_id,score,username,game_number,user_number
0,5c0964d7933dea443f61b922,NASA,10,user1,1,1
1,5c0964d7933dea443f61b923,NASA,7,user2,1,2
2,5c0964d7933dea443f61b924,NASA,2,user3,1,4
3,5c0964d7933dea443f61b925,STAR WARS,5,user2,2,2
4,5c0964d7933dea443f61b926,STAR WARS,10,user4,2,5
5,5c0964d7933dea443f61b927,STAR WARS,2,user5,2,3
6,5c0964d7933dea443f61b928,Mario,10,user1,0,1
7,5c0964d7933dea443f61b929,Mario,8,user3,0,4
8,5c0964d7933dea443f61b92a,Mario,2,user6,0,0


In [26]:
for game in df.game_id.unique():
    print(game, 
          (df['game_id']==game).sum())

NASA 3
STAR WARS 3
Mario 3


This for loop below successfully drops any rows where the game has exactly 3 reviews
# Note: this for loop is not efficient at all when dealing with huge dataframes

In [27]:
for game in df.game_id.unique():
    if (df.game_id==game).sum() ==3:
        df.drop(df[df.game_id==game].index, inplace=True)

In [28]:
df.drop(df[df.game_id=='NASA'].index, inplace=True)

In [29]:
df

,_id,game_id,score,username,game_number,user_number


Attempt at the same thing as above but by using groupby

In [76]:
user_group = pd.DataFrame(df.groupby('username').size(), columns=['username_count'])#.reset_index()#name='user_number')

In [77]:
user_group

,username_count
username,
user1,2
user2,2
user3,2
user4,1
user5,1
user6,1


In [82]:
df = df.join(user_group, on='username')

In [86]:
df.loc[df.username_count>1, :]

,_id,game_id,score,username,username_count
0,5c0964d7933dea443f61b922,NASA,10,user1,2
1,5c0964d7933dea443f61b923,NASA,7,user2,2
2,5c0964d7933dea443f61b924,NASA,2,user3,2
3,5c0964d7933dea443f61b925,STAR WARS,5,user2,2
6,5c0964d7933dea443f61b928,Mario,10,user1,2
7,5c0964d7933dea443f61b929,Mario,8,user3,2


In [87]:
df[df.username_count>1]

,_id,game_id,score,username,username_count
0,5c0964d7933dea443f61b922,NASA,10,user1,2
1,5c0964d7933dea443f61b923,NASA,7,user2,2
2,5c0964d7933dea443f61b924,NASA,2,user3,2
3,5c0964d7933dea443f61b925,STAR WARS,5,user2,2
6,5c0964d7933dea443f61b928,Mario,10,user1,2
7,5c0964d7933dea443f61b929,Mario,8,user3,2
